In [1]:
!python -c "import sys; print(sys.executable)"

/home/muhammadhuzaifawaseem/anaconda3/bin/python


# MLFlow lab

In [2]:
import pandas as pd

In [3]:
pd.__version__

'2.0.0'

### Setting up MLFlow tracking server

We also specify artifact root and backend store URI. This makes it possible to store models.

After running this command tracking server will be accessible at `localhost:5000`

In [4]:
%%bash --bg

mlflow server --host 0.0.0.0 \
    --port 5000 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./mlruns

### MLProject file

This file is used to configure MLFlow steps.

Using `MLproject` we can define our project's pipeline steps, called *entry points*.

Each entry point in this file corresponds to a shell command.

Entry points can be ran using

```
mlflow run -e <ENTRY_POINT>
```

By default `mlflow run` runs `main` entrypoint.

In [5]:
%cat MLproject

name: basic_mlflow

# this file is used to configure Python package dependencies.
# it uses Anaconda, but it can be also alternatively configured to use pip.
conda_env: conda.yaml

# entry points can be ran using `mlflow run <project_name> -e <entry_point_name>
entry_points:
  download_data:
    # you can run any command using MLFlow
    command: "bash download_data.sh"
  # MLproject file has to have main entry_point. It can be toggled without using -e option.

  main:
    # parameters is a key-value collection.
    parameters:
      file_name:
        type: str
        default: "winequality-red.csv"
      max_n:
        type: int
        default: 2
      max_d:
        type: int
        default: 2
    command: "python train.py {file_name} {max_n} {max_d}"

First we need to download data. We will use weather data from previous machine learning tutorial.

In [6]:
%%bash
source mlflow_env_vars.sh
mlflow run .  -e download_data

2023/05/05 15:26:23 INFO mlflow.utils.conda: Conda environment mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 already exists.
2023/05/05 15:26:23 INFO mlflow.projects.utils: === Created directory /tmp/tmp5k2zcz9s for downloading remote URIs passed to arguments of type 'path' ===
2023/05/05 15:26:23 INFO mlflow.projects.backend.local: === Running command 'source /home/muhammadhuzaifawaseem/anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 1>&2 && bash download_data.sh' in run with ID '12a1afe171614c9a90582265262d44f4' === 
File ‘winequality-red.csv’ already there; not retrieving.

2023/05/05 15:26:24 INFO mlflow.projects: === Run (ID '12a1afe171614c9a90582265262d44f4') succeeded ===


## Training

Now we can train models. See `train.py`.
It contains code from supervised machine learning tutorial; we added tracking metrics and model.

We will train kNN models for $k \in \{1, 2, ..., 10\}$ using *temperature* and *casual* features.

After running this command you can go to `localhost:5000` and see the trained models.

In [7]:
import sklearn

In [8]:
sklearn.__version__

'1.2.2'

In [9]:
!python --version

Python 3.10.9


In [10]:
%%bash
source mlflow_env_vars.sh
mlflow run . 

2023/05/05 15:26:28 INFO mlflow.utils.conda: Conda environment mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 already exists.
2023/05/05 15:26:28 INFO mlflow.projects.utils: === Created directory /tmp/tmplugt2ud4 for downloading remote URIs passed to arguments of type 'path' ===
2023/05/05 15:26:28 INFO mlflow.projects.backend.local: === Running command 'source /home/muhammadhuzaifawaseem/anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33 1>&2 && python train.py winequality-red.csv 2 2' in run with ID '5ff25d961b5b403a85b00b0fe848db7e' === 
/home/muhammadhuzaifawaseem/anaconda3/envs/mlflow-dd0fbdd40ba98798131458f29496394bd1a3fb33/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'sklearn_RandomForest' already exists. Creating a new version of this model...
2023/05/05 15:26:32 INFO mlflow.tracking._model

## Inspecting stored models

The trained models are stored in `mlruns/0`.

These directories contain artifacts and config that is needed to serve them.

In [11]:
%%bash
last_model_path=$(ls -tr mlruns/0/ | tail -1)
cat mlruns/0/$last_model_path/artifacts/RandomForest/MLmodel

artifact_path: RandomForest
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.11.3
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.2.2
mlflow_version: 2.3.1
model_uuid: ba14cfbbacc947f8a80ce7c4b8597351
run_id: e2fe8645611747b9b2fb5f30f5de380a
utc_time_created: '2023-05-05 05:05:28.533741'


In [12]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [13]:

import mlflow

In [14]:
mlflow.__version__

'2.3.1'

## Serving model

Now that we trained our models we can go to *Models* page on MLFLow UI (http://localhost:5000/#/models).

Click *sklearn_knn* on this page, choose a model and move it to *Production* stage.

The following cell will serve the model at localhost on port 5001.

In [15]:
%%bash --bg
source mlflow_env_vars.sh
mlflow --version
mlflow models serve -m models:/sklearn_RandomForest/Production -p 5001 --env-manager=conda 


# Prediction

We'll load data that we can feed into prediction server.

In [16]:
df = pd.read_csv("winequality-red.csv", sep=';')[["alcohol","pH", "quality"]]
df["is_quality"] = df["quality"] == 6

df[~df["is_quality"]].head()

alcohol    pH  quality  is_quality
0      9.4  3.51        5       False
1      9.8  3.20        5       False
2      9.8  3.26        5       False
4      9.4  3.51        5       False
5      9.4  3.51        5       False

In [17]:
%%bash
data='[[7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4], [7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8]]'
echo $data

curl -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4], [7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   135  100    23  100   112  11698  56968 --:--:-- --:--:-- --:--:--  131k


{"predictions": [5, 5]}

In [18]:
%%bash
data='[[7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4], [7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8]]'
echo $data

curl -d "{\"instances\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4], [7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   138  100    23  100   115  10074  50372 --:--:-- --:--:-- --:--:-- 69000


{"predictions": [5, 5]}

In [19]:
%%bash
data='[[7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4], [7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8]]'
columns='["temp","casual"]'
echo $data

curl -d "{\"dataframe_split\":{\"columns\":[\"temp\",\"casual\"],\"data\": $data}}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4], [7.8,0.88,0,2.6,0.098,25,67,0.9968,3.2,0.68,9.8]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   345  100   187  100   158   188k   159k --:--:-- --:--:-- --:--:--  336k


{"error_code": "BAD_REQUEST", "message": "Provided dataframe_split field is not a valid dataframe representation in 'split' format. Error: '2 columns passed, passed data had 11 columns'"}

Voila! We see that the model outputs correct predictions.